In [27]:
import json
from openai import OpenAI
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from elasticsearch import Elasticsearch

In [28]:
client =  Groq(api_key = os.environ['GROQ_API_KEY'])


In [29]:
with open('../data/vietnamese_rag/documents.json', 'rt') as f_in:
    documents = json.load(f_in)

In [30]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [31]:
len(model.encode("This is a simple sentence"))

384

In [32]:
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '220d97b49b71', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Bjagydp1SHuMN5WDZF6iog', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [33]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "context": {"type": "text"},
            "question": {"type": "text"},
            "answer": {"type": "text"}, 
            "group": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
        }
    }
}

In [34]:
index_name = "vietnamese-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vietnamese-questions'})

In [35]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector1.pkl', 'rb') as file:
    operations1 = pickle.load(file)

In [36]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector2.pkl', 'rb') as file:
    operations2 = pickle.load(file)

In [37]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector3.pkl', 'rb') as file:
    operations3 = pickle.load(file)

In [38]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector4.pkl', 'rb') as file:
    operations4 = pickle.load(file)

In [39]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector5.pkl', 'rb') as file:
    operations5 = pickle.load(file)

In [40]:
for doc in tqdm(operations1):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 1217/1217 [00:28<00:00, 43.32it/s]


In [42]:
for doc in tqdm(operations2):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 1217/1217 [00:25<00:00, 47.85it/s]


In [43]:
for doc in tqdm(operations3):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 1217/1217 [00:25<00:00, 48.05it/s]


In [44]:
for doc in tqdm(operations4):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 2028/2028 [00:42<00:00, 47.73it/s]


In [45]:
for doc in tqdm(operations5):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 1221/1221 [00:25<00:00, 47.09it/s]


In [54]:
search_term = "hoa hậu thế giới có xinh không ?"
vector_search_term = model.encode(search_term)

In [55]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [56]:
res = es_client.search(index=index_name, knn=query, source=[ "answer", "question", "group"])
res["hits"]["hits"]

[{'_index': 'vietnamese-questions',
  '_id': 'S7QbLJEBFdjf_YyEmVWJ',
  '_score': 0.7489821,
  '_source': {'question': "Lê Thanh Tú có thể đạt thành tích gì trong cuộc thi 'Hoa hậu hoàn vũ Việt Nam'?",
   'answer': "Lê Thanh Tú hiện tại là một trong những thí sinh đạt được số điểm rất cao trong phần thi 'Tôi là Hoa hậu hoàn vũ Việt Nam', vượt xa cả hai chân dài đình đám Mâu Thủy và Hoàng Thùy.",
   'group': 'General'}},
 {'_index': 'vietnamese-questions',
  '_id': 'DLQbLJEBFdjf_YyE_lqa',
  '_score': 0.7489821,
  '_source': {'question': "Lê Thanh Tú có thể đạt thành tích gì trong cuộc thi 'Hoa hậu hoàn vũ Việt Nam'?",
   'answer': "Lê Thanh Tú hiện tại là một trong những thí sinh đạt được số điểm rất cao trong phần thi 'Tôi là Hoa hậu hoàn vũ Việt Nam', vượt xa cả hai chân dài đình đám Mâu Thủy và Hoàng Thùy.",
   'group': 'General'}},
 {'_index': 'vietnamese-questions',
  '_id': 'ybQbLJEBFdjf_YyEjlQH',
  '_score': 0.73057187,
  '_source': {'question': 'Cựu Hoa hậu Việt Nam Kỳ Duyên đang

In [71]:

import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['group']}-{doc['question']}-{doc['context']}-{doc['answer']}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [72]:

for doc in documents:
    doc['id'] = generate_document_id(doc)

In [77]:
documents[0]

{'group': 'General',
 'context': 'ngữ cảnh 1: Vượt qua nỗi sợ hãi, Minh Tú xuất sắc vào chung kết Asia\'s Next Top Model.  Bỏ qua hết những lo lắng về căn bệnh tim, Minh Tú đã hết sức chế ngự sự sợ hãi và hoàn thành tốt phần thử thách đi catwalk khi bị treo lơ lửng trên một tòa nhà cao tầng và dành chiếc vé vào đêm chung kết Asia’s Next Top Model..  Tối ngày 21/6, tập 12 của chương trình Asia’ Next Top Model mùa thứ 5 đã được lên sóng. 4 thí sinh cùng giám khảo bí mật Xiao Qing bước vào phần thi thử thách đặc biệt là đi catwalk khi bị treo lơ lửng trên một tòa nhà cao tầng. Các cô gái phải vừa tạo dáng vừa thể hiện được sự mạnh mẽ, quyến rũ riêng.. Mặc dù có bệnh về tim nhưng Minh Tú đã cố gắng hết sức chế ngự sự sợ hãi và hoàn thành tốt phần thử thách này. Thí sinh chiến thắng là Shikin và đại diện Việt Nam cũng không kém cạnh gì khi xuất sắc xếp ở vị trí thứ 2, báo GĐVN đưa tin.. Bước vào thử thách, Minh Tú hét lớn: “Tôi còn nhiều việc chưa làm được đâu” vì sợ sẽ rơi xuống tuy nhiên 

In [80]:

with open('../data/vietnamese_rag/documents-with-ids1.json', 'wt') as f_out:
    json.dump(documents[:1217] , f_out, indent=2)

In [81]:
with open('../data/vietnamese_rag/documents-with-ids2.json', 'wt') as f_out:
    json.dump(documents[1217:1217*2] , f_out, indent=2)

In [82]:
with open('../data/vietnamese_rag/documents-with-ids3.json', 'wt') as f_out:
    json.dump(documents[1217*2:1217*3] , f_out, indent=2)

In [83]:
with open('../data/vietnamese_rag/documents-with-ids4.json', 'wt') as f_out:
    json.dump(documents[1217*3:1217*4] , f_out, indent=2)

In [84]:
with open('../data/vietnamese_rag/documents-with-ids5.json', 'wt') as f_out:
    json.dump(documents[1217*4] , f_out, indent=2)